In [1]:
from sklearn import model_selection, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
file_real = open('real/sam_content.txt', 'r', encoding='utf-8')
file_fake = open('fake/sam_content_phoney.txt', 'r', encoding='utf-8')

In [4]:
data = []
labels = []
for line in file_real:
    data.append(line)
    labels.append(1)
for line in file_fake:
    data.append(line)
    labels.append(0)

In [5]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data, labels, test_size=.3)

In [7]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [17]:
pi = [ sum(train_y)/len(train_y), sum([1 for x in train_y if x == 0])/len(train_y)]
print(pi)

[0.4948554630083293, 0.5051445369916707]


In [18]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_x)
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x).todense()
xvalid_count =  count_vect.transform(valid_x).todense()

In [19]:
wordFreq = pd.DataFrame(columns=['words','class1','class2'])
wordFreq['words'] = count_vect.get_feature_names()

x_train_class1 = xtrain_count[train_y==0]
x_train_class2 = xtrain_count[train_y==1]

count_class1 = np.sum(x_train_class1,axis=0)
count_class2 = np.sum(x_train_class2,axis=0)

vocab_size1 = len(np.where(count_class1==0)[1])
vocab_size2 = len(np.where(count_class2==0)[1])

alpha=10
count_class1 = np.array( (count_class1+alpha) /(np.sum(count_class1)+vocab_size1 +1))
count_class2 = np.array( (count_class2+alpha) /(np.sum(count_class2)+vocab_size2 +1))

wordFreq['class1'] = pd.Series(count_class1.ravel())
wordFreq['class2'] = pd.Series(count_class2.ravel())

In [20]:
train_preds = np.zeros(len(xtrain_count))
for i in range(len(xtrain_count)):
    idx = np.where(xtrain_count[i,:]!=0)[1]
    lh1 = wordFreq['class1'].iloc[idx].prod()
    lh2 = wordFreq['class2'].iloc[idx].prod()
    posterior1 = lh1*pi[0]
    posterior2 = lh2*pi[1]

    if posterior1>posterior2:
        train_preds[i] = 0
    else:
        train_preds[i] = 1


matches = np.sum(train_y==train_preds)
print('Train accuracy is: ' + str(matches/len(train_preds)))

Train accuracy is: 0.5634492895639392


In [21]:
valid_preds = np.zeros(len(xvalid_count))
for i in range(len(xvalid_count)):
    idx = np.where(xvalid_count[i,:]!=0)[1]
    lh1 = wordFreq['class1'].iloc[idx].prod()
    lh2 = wordFreq['class2'].iloc[idx].prod()
    posterior1 = lh1*pi[0]
    posterior2 = lh2 * pi[1]

    if posterior1>posterior2:
        valid_preds[i] = 0
    else:
        valid_preds[i] = 1
    temp = 1

matches = np.sum(valid_y==valid_preds)
print('Validation accuracy is: '+str(matches/len(valid_preds)))

Validation accuracy is: 0.5901826484018264
